# R

In [20]:
library(data.table)

# Read the CSV
df <- fread("/Users/franco.benites/dm2025b/src/workflows/experiments_tracking.csv")

In [22]:
colnames(df)

[1] "Experimento"      "orden corrida"    "date"             "publicScore"     
 [5] "envios"           "semilla"          "num_leaves"       "min_data_in_leaf"
 [9] "feature_fraction" "learning_rate"    "num_iterations"   "drift"

In [32]:
# Check which column index "orden corrida" is (should be column 2)
print(colnames(df)[2])

# Rename by index
setnames(df, old = 2, new = "orden_corrida")

# Verify the change
colnames(df)

[1] "orden_corrida"


[1] "Experimento"      "orden_corrida"    "date"             "publicScore"     
 [5] "envios"           "semilla"          "num_leaves"       "min_data_in_leaf"
 [9] "feature_fraction" "learning_rate"    "num_iterations"   "drift"

In [23]:
df[, drift := tolower(drift)]
df[, drift := gsub(" ", "_", drift)]

In [24]:
df <- df[!is.na(df$publicScore)]

In [25]:
# Print head of data
head(df)

# Get unique drift methods
drift_methods <- unique(df$drift)
print(drift_methods)

# Create separate vectors for each drift method
score_by_drift <- list(
  ipc = df[drift == "ipc", publicScore],
  nucleo = df[drift == "nucleo", publicScore],
  dolar_blue = df[drift == "dolar_blue", publicScore],
  ninguno = df[drift == "ninguno", publicScore],
  rank_simple = df[drift == "rank_simple", publicScore]
)


Experimento,orden corrida,date,publicScore,envios,semilla,num_leaves,min_data_in_leaf,feature_fraction,learning_rate,num_iterations,drift
<int>,<int>,<dttm>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>
9116,1,NA,6.435,2400,100379,60,2100,1,0.1,1032,dolar_blue
9117,2,NA,6.505,1800,101111,202,4281,1,0.1,215,dolar_blue
9118,3,2025-09-12 17:35:58,7.506,1800,250007,120,8185,1,0.1,1382,dolar_blue
9119,4,NA,7.336,1800,700001,220,8190,1,0.1,482,dolar_blue
9120,5,NA,6.055,NA,800011,NA,NA,1,0.1,NA,dolar_blue
9131,6,NA,6.575,NA,237689,NA,NA,1,0.1,NA,dolar_blue


[1] "dolar_blue"     "ipc"            "ninguno"        "nucleo"        
[5] "rank__simple"   "rank_cero_fijo" "std"           


In [26]:
wt <- wilcox.test(score_by_drift$ipc, score_by_drift$nucleo)
pvalue <- wt$p.value
out <- 0
if (pvalue < 0.05 && mean(score_by_drift$ipc) > mean(score_by_drift$nucleo)) {
  out <- 1
} else if (pvalue < 0.05 && mean(score_by_drift$ipc) < mean(score_by_drift$nucleo)) {
  out <- 2
}
print(out)

[1] 0


In [34]:
# Get all unique pairs of methods
methods <- names(score_by_drift)
pairs <- expand.grid(method1 = methods, method2 = methods, stringsAsFactors = FALSE)  # Prevent factor conversion
pairs <- pairs[pairs$method1 < pairs$method2, ]  # Keep only unique pairs

# Function to compare two methods
compare_methods <- function(method1, method2) {
  # Convert to numeric if needed
  scores1 <- as.numeric(score_by_drift[[method1]])
  scores2 <- as.numeric(score_by_drift[[method2]])
  
  wt <- wilcox.test(scores1, scores2)
  pvalue <- wt$p.value
  
  out <- "NOT-SIGNIFICANT"
  if (pvalue < 0.05 && mean(scores1) > mean(scores2)) {
    out <- "1 WINS"  # method1 significantly better
  } else if (pvalue < 0.05 && mean(scores1) < mean(scores2)) {
    out <- "2 WINS"  # method2 significantly better
  }
  
  return(data.frame(
    method1 = method1,
    method2 = method2,
    pvalue = pvalue,
    mean1 = mean(scores1),
    mean2 = mean(scores2),
    result = out
  ))
}

# Apply comparison to all pairs
results <- do.call(rbind, 
  lapply(1:nrow(pairs), function(i) {
    compare_methods(pairs$method1[i], pairs$method2[i])
  })
)

# Print results in a nice format
print(results)

Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1, scores2):
"cannot compute exact p-value with ties"
Warning me

          method1        method2      pvalue   mean1   mean2          result
1      dolar_blue            ipc 0.289737063 6.76920 7.08340 NOT-SIGNIFICANT
2             ipc         nucleo 0.594072594 7.08340 6.79300 NOT-SIGNIFICANT
3      dolar_blue         nucleo 1.000000000 6.76920 6.79300 NOT-SIGNIFICANT
4         ninguno         nucleo 0.434554664 7.03285 6.79300 NOT-SIGNIFICANT
5             ipc        ninguno 1.000000000 7.08340 7.03285 NOT-SIGNIFICANT
6      dolar_blue        ninguno 0.179508759 6.76920 7.03285 NOT-SIGNIFICANT
7             ipc    rank_simple 0.018543376 7.08340 6.05650          1 WINS
8          nucleo    rank_simple 0.111027488 6.79300 6.05650 NOT-SIGNIFICANT
9      dolar_blue    rank_simple 0.123005477 6.76920 6.05650 NOT-SIGNIFICANT
10        ninguno    rank_simple 0.007269502 7.03285 6.05650          1 WINS
11 rank_cero_fijo    rank_simple 0.012890679 6.93265 6.05650          1 WINS
12            ipc rank_cero_fijo 0.964901275 7.08340 6.93265 NOT-SIGNIFICANT

In [28]:
# Print head of data
head(df)

# Get unique drift methods
drift_methods <- unique(df$drift)
print(drift_methods)

# Create separate vectors for each drift method that actually exists
score_by_drift <- list()

# Only create entries for methods that exist in the data
if("ipc" %in% drift_methods) {
  score_by_drift$ipc <- df[drift == "ipc", publicScore]
}
if("nucleo" %in% drift_methods) {
  score_by_drift$nucleo <- df[drift == "nucleo", publicScore]
}
if("dolar_blue" %in% drift_methods) {
  score_by_drift$dolar_blue <- df[drift == "dolar_blue", publicScore]
}
if("ninguno" %in% drift_methods) {
  score_by_drift$ninguno <- df[drift == "ninguno", publicScore]
}
if("rank__simple" %in% drift_methods) {  # Note: double underscore
  score_by_drift$rank_simple <- df[drift == "rank__simple", publicScore]
}
if("rank_cero_fijo" %in% drift_methods) {
  score_by_drift$rank_cero_fijo <- df[drift == "rank_cero_fijo", publicScore]
}
if("std" %in% drift_methods) {
  score_by_drift$std <- df[drift == "std", publicScore]
}

# Check what we actually have
print(names(score_by_drift))
print(sapply(score_by_drift, length))

Experimento,orden corrida,date,publicScore,envios,semilla,num_leaves,min_data_in_leaf,feature_fraction,learning_rate,num_iterations,drift
<int>,<int>,<dttm>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>
9116,1,NA,6.435,2400,100379,60,2100,1,0.1,1032,dolar_blue
9117,2,NA,6.505,1800,101111,202,4281,1,0.1,215,dolar_blue
9118,3,2025-09-12 17:35:58,7.506,1800,250007,120,8185,1,0.1,1382,dolar_blue
9119,4,NA,7.336,1800,700001,220,8190,1,0.1,482,dolar_blue
9120,5,NA,6.055,NA,800011,NA,NA,1,0.1,NA,dolar_blue
9131,6,NA,6.575,NA,237689,NA,NA,1,0.1,NA,dolar_blue


[1] "dolar_blue"     "ipc"            "ninguno"        "nucleo"        
[5] "rank__simple"   "rank_cero_fijo" "std"           
[1] "ipc"            "nucleo"         "dolar_blue"     "ninguno"       
[5] "rank_simple"    "rank_cero_fijo" "std"           
           ipc         nucleo     dolar_blue        ninguno    rank_simple 
            10              5             10             20             10 
rank_cero_fijo            std 
            20             10 


In [33]:
# First, let's create a helper function to get the orden_corrida for each drift method
get_ordered_scores <- function(drift_method) {
  # Get the subset for this drift method
  subset_df <- df[drift == drift_method]
  
  # Order by orden_corrida
  setorder(subset_df, orden_corrida)
  
  # Return the ordered publicScore
  return(subset_df$publicScore)
}

# Create ordered score vectors
score_by_drift_ordered <- list()

for(method in drift_methods) {
  if(method %in% df$drift) {
    score_by_drift_ordered[[method]] <- get_ordered_scores(method)
  }
}

# Or more concisely using lapply
score_by_drift_ordered <- lapply(drift_methods, function(method) {
  subset_df <- df[drift == method]
  setorder(subset_df, orden_corrida) 
  return(subset_df$publicScore)
})
names(score_by_drift_ordered) <- drift_methods

In [37]:
library(data.table)

# Read the CSV
df <- fread("/Users/franco.benites/dm2025b/src/workflows/experiments_tracking.csv")

# Rename column
setnames(df, old = 2, new = "orden_corrida")

# Clean drift column
df[, drift := tolower(drift)]
df[, drift := gsub(" ", "_", drift)]

# Remove rows with missing publicScore
df <- df[!is.na(publicScore)]

# Order the entire dataset by orden_corrida
setorder(df, orden_corrida)

# Print head of data
head(df)

# Get unique drift methods
drift_methods <- unique(df$drift)
print(drift_methods)

# Create ordered score vectors for each drift method
get_ordered_scores <- function(drift_method) {
  subset_df <- df[drift == drift_method]
  setorder(subset_df, orden_corrida)  # Ensure ordering within each method
  return(subset_df$publicScore)
}

# Create score_by_drift with proper ordering
score_by_drift <- list()
for(method in drift_methods) {
  if(method %in% df$drift) {
    score_by_drift[[method]] <- get_ordered_scores(method)
  }
}

# Print what we have
print(names(score_by_drift))
print(sapply(score_by_drift, length))

# Updated comparison function with paired tests using shortest length
compare_methods <- function(method1, method2) {
  scores1 <- as.numeric(score_by_drift[[method1]])
  scores2 <- as.numeric(score_by_drift[[method2]])
  
  # Get the minimum length to ensure paired comparison
  min_length <- min(length(scores1), length(scores2))
  
  # Truncate both vectors to the same length
  scores1_paired <- scores1[1:min_length]
  scores2_paired <- scores2[1:min_length]
  
  # Always use paired test now
  wt <- wilcox.test(scores1_paired, scores2_paired, paired = TRUE)
  
  pvalue <- wt$p.value
  
  out <- "NOT-SIGNIFICANT"
  if (pvalue < 0.05 && mean(scores1_paired) > mean(scores2_paired)) {
    out <- paste(method1, "WINS")
  } else if (pvalue < 0.05 && mean(scores1_paired) < mean(scores2_paired)) {
    out <- paste(method2, "WINS")
  }
  
  return(data.frame(
    method1 = method1,
    method2 = method2,
    n_compared = min_length,  # Show how many pairs were compared
    pvalue = round(pvalue, 6),
    mean1 = round(mean(scores1_paired), 3),
    mean2 = round(mean(scores2_paired), 3),
    diff = round(mean(scores1_paired) - mean(scores2_paired), 3),
    result = out,
    stringsAsFactors = FALSE
  ))
}

# The rest of the code remains the same...
# Get all unique pairs and run comparisons
methods <- names(score_by_drift)
pairs <- expand.grid(method1 = methods, method2 = methods, stringsAsFactors = FALSE)
pairs <- pairs[pairs$method1 < pairs$method2, ]

# Run all pairwise comparisons
results <- do.call(rbind, 
  lapply(1:nrow(pairs), function(i) {
    compare_methods(pairs$method1[i], pairs$method2[i])
  })
)

# Sort results by p-value to see most significant first
setorder(results, pvalue)

# Print results
print(results)

# Summary statistics by method
summary_stats <- df[, .(
  count = .N,
  mean_score = round(mean(publicScore, na.rm = TRUE), 3),
  median_score = round(median(publicScore, na.rm = TRUE), 3),
  sd_score = round(sd(publicScore, na.rm = TRUE), 3),
  min_score = round(min(publicScore, na.rm = TRUE), 3),
  max_score = round(max(publicScore, na.rm = TRUE), 3)
), by = drift]

setorder(summary_stats, -mean_score)
print("\nSummary by drift method:")
print(summary_stats)

# Find best performing method
best_method <- summary_stats[1, drift]
cat("\nBest performing method:", best_method, "with mean score:", summary_stats[1, mean_score], "\n")

# Save results
fwrite(results, "pairwise_comparisons.csv")
fwrite(summary_stats, "method_summary.csv")

Experimento,orden_corrida,date,publicScore,envios,semilla,num_leaves,min_data_in_leaf,feature_fraction,learning_rate,num_iterations,drift
<int>,<int>,<dttm>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>
9116,1,NA,6.435,2400,100379,60,2100,1.0000000,0.10000000,1032,dolar_blue
9105,1,NA,7.506,1900,100379,256,4940,1.0000000,0.10000000,1139,ipc
9203,1,2025-09-11 23:22:51,6.885,1900,100379,189,8188,1.0000000,0.10000000,502,ninguno
9111,1,NA,6.225,1800,100379,256,2967,1.0000000,0.10000000,746,nucleo
9208,1,2025-09-12 08:28:59,3.873,1800,100379,168,27,1.0000000,0.10000000,1367,rank__simple
9218,1,2025-09-14 03:07:27,7.045,2400,100379,149,1256,0.2886525,0.01406128,386,rank_cero_fijo


[1] "dolar_blue"     "ipc"            "ninguno"        "nucleo"        
[5] "rank__simple"   "rank_cero_fijo" "std"           
[1] "dolar_blue"     "ipc"            "ninguno"        "nucleo"        
[5] "rank__simple"   "rank_cero_fijo" "std"           
    dolar_blue            ipc        ninguno         nucleo   rank__simple 
            10             10             20              5             10 
rank_cero_fijo            std 
            20             10 


Warning message in wilcox.test.default(scores1_paired, scores2_paired, paired = TRUE):
"cannot compute exact p-value with ties"
Warning message in wilcox.test.default(scores1_paired, scores2_paired, paired = TRUE):
"cannot compute exact p-value with ties"


          method1        method2 n_compared   pvalue mean1 mean2   diff
15   rank__simple rank_cero_fijo         10 0.005859 6.056 7.295 -1.239
17            ipc            std         10 0.005859 7.083 5.211  1.873
21 rank_cero_fijo            std         10 0.005859 7.295 5.211  2.085
8             ipc   rank__simple         10 0.013672 7.083 6.056  1.027
11     dolar_blue rank_cero_fijo         10 0.019531 6.769 7.295 -0.526
18        ninguno            std         10 0.037109 7.017 5.211  1.807
9         ninguno   rank__simple         10 0.059171 7.017 6.056  0.961
10         nucleo   rank__simple          5 0.062500 6.793 5.810  0.983
16     dolar_blue            std         10 0.064453 6.769 5.211  1.558
7      dolar_blue   rank__simple         10 0.105469 6.769 6.056  0.713
5             ipc         nucleo          5 0.125000 7.502 6.793  0.709
14         nucleo rank_cero_fijo          5 0.125000 6.793 7.241 -0.448
1      dolar_blue            ipc         10 0.232422 6.769 7.083